In [2]:
library(tidyverse)
library(dplyr)

In [3]:
screening_info = read.csv("image_constructs_association_Cterm.csv")
#screening_info = screening_info[screening_info$Tagging =="C",]
head(screening_info)

,Condition,Image_ID,Rep,Construct,Tagging,GeneID,Species,Gene,Dup_position,Pos,Species_position
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A)
2,MTX,d000263_300_002,1,pKB11,C,2,K.africana,KAFR0K00320,DupA,A,Z.rouxii(Anc)
3,MTX,d000263_300_003,1,pKB11,C,3,T.blattae,TBLA0A02040,DupA,A,N.dairenensis(A)
4,MTX,d000263_300_004,1,pKB11,C,4,S.kudriavzevii,Skud_16.75,DupA,A,K.africana(A)
5,MTX,d000263_300_005,1,pKB11,C,5,K.africana,KAFR0C02290,DupB,B,C.glabrata(B)
6,MTX,d000263_300_006,1,pKB11,C,6,T.blattae,TBLA0B05310,DupB,B,T.phaffii(B)


In [4]:
miniarray_info = read.csv("miniarray_information_HRR25screening.csv")
# This file has MAM1 added in C24 R7 instead of HEM12
head(miniarray_info)

,Gene,ORF,c1536,r1536
,<chr>,<chr>,<int>,<int>
1,Border,Border,1,1
2,Border,Border,1,3
3,Border,Border,1,5
4,Border,Border,1,7
5,Border,Border,1,9
6,Border,Border,1,11


In [5]:
MTX_data = read.csv("2022-12-09_MTX_Sel2_AUC_data_Cterm.csv")
head(MTX_data)

,row,column,spot_well,Plate_ID,Selection,auc
,<int>,<int>,<chr>,<int>,<chr>,<dbl>
1,1,7,A7,11,MTX_2,89242.5
2,1,9,A9,11,MTX_2,94136.0
3,1,25,A25,11,MTX_2,88689.0
4,1,29,A29,11,MTX_2,85920.0
5,1,12,A12,11,MTX_2,89690.0
6,1,15,A15,11,MTX_2,93944.0


In [6]:
Diploid_data = read.csv("diploid_selection2_20221001_pyphe_output.csv")
head(Diploid_data)

,X,area,centroid,mean_intensity,perimeter,row,column,circularity,plate_number,spot_well,RA_well
,<int>,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>
1,1,3171,"(34.07568590350047, 1084.6701356039105)",0.8208408,212.4092,1,13,0.8832011,IMG_00065,A13,A7
2,2,3064,"(33.72682767624021, 1530.2519582245432)",0.8221190,206.7523,1,18,0.9007368,IMG_00065,A18,A9
3,3,3151,"(34.15677562678515, 1799.2926055220564)",0.8217584,211.8234,1,21,0.8824914,IMG_00065,A21,A11
4,4,3094,"(33.75371687136393, 2863.8212669683257)",0.8194557,209.0955,1,33,0.8892850,IMG_00065,A33,A17
5,5,3183,"(34.921143575243484, 3483.1555136663524)",0.8207068,212.4092,1,40,0.8865434,IMG_00065,A40,A20
6,6,3780,"(37.60079365079365, 287.53068783068784)",0.8146094,232.5513,1,4,0.8783430,IMG_00065,A4,A2


### STEPS

##### Idenfity samples
- Define unique ID for the samples; Condition.Construct.GeneID.Rep.Row.Col
- Define unique ID in each of the PPI data

##### Call growth on control
1. Add area of diploid in the MTX_data. Add by Unique ID

##### Parse gene information
1. Based on the row/col information, add gene information (GeneID/ORF)
2. Remove border controls
3. Reshape dataframe so we have basic information, and all the miniarray genes (or ORFs) as columns, and in each row we have different HRR25 constructs



In [7]:
screening_info$Construct_ID =  paste(screening_info$Construct,screening_info$Tagging,screening_info$GeneID,sep=".")
screening_info$Plate_ID = as.numeric(substr(screening_info$Image_ID,13,15))
head(screening_info)

,Condition,Image_ID,Rep,Construct,Tagging,GeneID,Species,Gene,Dup_position,Pos,Species_position,Construct_ID,Plate_ID
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,1
2,MTX,d000263_300_002,1,pKB11,C,2,K.africana,KAFR0K00320,DupA,A,Z.rouxii(Anc),pKB11.C.2,2
3,MTX,d000263_300_003,1,pKB11,C,3,T.blattae,TBLA0A02040,DupA,A,N.dairenensis(A),pKB11.C.3,3
4,MTX,d000263_300_004,1,pKB11,C,4,S.kudriavzevii,Skud_16.75,DupA,A,K.africana(A),pKB11.C.4,4
5,MTX,d000263_300_005,1,pKB11,C,5,K.africana,KAFR0C02290,DupB,B,C.glabrata(B),pKB11.C.5,5
6,MTX,d000263_300_006,1,pKB11,C,6,T.blattae,TBLA0B05310,DupB,B,T.phaffii(B),pKB11.C.6,6


In [8]:
#MTX_data$Image_ID = as.numeric(substr(MTX_data$plate_number,1,15))

screening_info_MTX = screening_info[screening_info$Condition=="MTX",]

#head(MTX_data)

MTX_data <- (merge(screening_info_MTX, MTX_data,  by=c('Plate_ID')))#,'Condition', 'Image_ID', 'Rep', 'Construct', 'Tagging', 'GeneID', 'Gene', 'Construct_ID')))

head(MTX_data)



,Plate_ID,Condition,Image_ID,Rep,Construct,Tagging,GeneID,Species,Gene,Dup_position,Pos,Species_position,Construct_ID,row,column,spot_well,Selection,auc
,<dbl>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>
1,1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,18,2,R2,MTX_2,50886.0
2,1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,18,1,R1,MTX_2,46501.5
3,1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,18,3,R3,MTX_2,3991.0
4,1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,18,5,R5,MTX_2,3617.0
5,1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,18,48,R48,MTX_2,53348.5
6,1,MTX,d000263_300_001,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,S.bayanusvar.uvarum(A),pKB11.C.1,18,47,R47,MTX_2,54706.0


In [9]:
MTX_data$UniqueID= paste(MTX_data$Construct_ID,paste("Rep",MTX_data$Rep,sep=""),paste("R",MTX_data$row,sep=""),paste("C",MTX_data$column,sep=""),sep=".")
MTX_data$position= paste( "Pos",paste("R",MTX_data$row,sep=""),paste("C",MTX_data$column,sep=""),sep=".")
MTX_data =  select(MTX_data, UniqueID,Construct_ID,Construct,Tagging,Rep,GeneID,Species,Gene,Species_position,Pos,row,column,position,auc)
head(MTX_data)

Diploid_data$Image_ID = Diploid_data$plate_number
screening_info_Dip = screening_info[screening_info$Condition=="Diploid",]
Diploid_data <- (merge(screening_info_Dip, Diploid_data, by = 'Image_ID'))
Diploid_data$UniqueID= paste(Diploid_data$Construct_ID,paste("Rep",Diploid_data$Rep,sep=""),paste("R",Diploid_data$row,sep=""),paste("C",Diploid_data$column,sep=""),sep=".")
head(Diploid_data)

,UniqueID,Construct_ID,Construct,Tagging,Rep,GeneID,Species,Gene,Species_position,Pos,row,column,position,auc
,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>
1,pKB11.C.1.Rep1.R18.C2,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,18,2,Pos.R18.C2,50886.0
2,pKB11.C.1.Rep1.R18.C1,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,18,1,Pos.R18.C1,46501.5
3,pKB11.C.1.Rep1.R18.C3,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,18,3,Pos.R18.C3,3991.0
4,pKB11.C.1.Rep1.R18.C5,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,18,5,Pos.R18.C5,3617.0
5,pKB11.C.1.Rep1.R18.C48,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,18,48,Pos.R18.C48,53348.5
6,pKB11.C.1.Rep1.R18.C47,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,18,47,Pos.R18.C47,54706.0


,Image_ID,Condition,Rep,Construct,Tagging,GeneID,Species,Gene,Dup_position,Pos,⋯,centroid,mean_intensity,perimeter,row,column,circularity,plate_number,spot_well,RA_well,UniqueID
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,IMG_00049,Diploid,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,⋯,"(2800.6563272028548, 2415.384298654955)",0.8169048,229.5807,32,28,0.8685566,IMG_00049,`28,P14,pKB11.C.1.Rep1.R32.C28
2,IMG_00049,Diploid,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,⋯,"(2799.1600609756097, 3041.049695121951)",0.8120576,215.6812,32,35,0.8860515,IMG_00049,`35,P18,pKB11.C.1.Rep1.R32.C35
3,IMG_00049,Diploid,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,⋯,"(2084.8730385164054, 3660.270090347123)",0.8155688,171.0955,24,42,0.9027598,IMG_00049,X42,L21,pKB11.C.1.Rep1.R24.C42
4,IMG_00049,Diploid,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,⋯,"(2800.967886065345, 2595.659592292656)",0.8164649,226.4092,32,30,0.8778619,IMG_00049,`30,P15,pKB11.C.1.Rep1.R32.C30
5,IMG_00049,Diploid,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,⋯,"(2085.0053242981608, 3566.874636979671)",0.8134597,169.7817,24,41,0.9006544,IMG_00049,X41,L21,pKB11.C.1.Rep1.R24.C41
6,IMG_00049,Diploid,1,pKB11,C,1,S.bayanusvar.uvarum,Suva_16.102,DupA,A,⋯,"(748.4432558139534, 2602.3562790697674)",0.8287205,171.4386,9,30,0.9192446,IMG_00049,I30,E15,pKB11.C.1.Rep1.R9.C30


In [10]:
Diploid_area =  select(Diploid_data, UniqueID, area)
names(Diploid_area)[2] = "Diploid_sel2_area"


In [11]:
MTX_data <- (merge(Diploid_area, MTX_data, by = 'UniqueID'))
head(MTX_data)

,UniqueID,Diploid_sel2_area,Construct_ID,Construct,Tagging,Rep,GeneID,Species,Gene,Species_position,Pos,row,column,position,auc
,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>
1,pKB11.C.1.Rep1.R1.C14,3003,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,1,14,Pos.R1.C14,81671.5
2,pKB11.C.1.Rep1.R1.C17,3051,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,1,17,Pos.R1.C17,83744.5
3,pKB11.C.1.Rep1.R1.C24,3126,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,1,24,Pos.R1.C24,86575.0
4,pKB11.C.1.Rep1.R1.C27,3112,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,1,27,Pos.R1.C27,78039.5
5,pKB11.C.1.Rep1.R1.C28,2855,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,1,28,Pos.R1.C28,82017.5
6,pKB11.C.1.Rep1.R1.C3,2870,pKB11.C.1,pKB11,C,1,1,S.bayanusvar.uvarum,Suva_16.102,S.bayanusvar.uvarum(A),A,1,3,Pos.R1.C3,79475.0


In [12]:
head(miniarray_info)

,Gene,ORF,c1536,r1536
,<chr>,<chr>,<int>,<int>
1,Border,Border,1,1
2,Border,Border,1,3
3,Border,Border,1,5
4,Border,Border,1,7
5,Border,Border,1,9
6,Border,Border,1,11


In [13]:
miniarray_info$position = paste("Pos",paste("R",miniarray_info$r1536,sep=""),paste("C",miniarray_info$c1536,sep=""),sep=".")

In [14]:
miniarray_pos_table =  select(miniarray_info, position,Gene, ORF)
names(miniarray_pos_table) = c("position","Miniarray.Gene","Miniarray.ORF")

In [15]:
head(miniarray_pos_table)

,position,Miniarray.Gene,Miniarray.ORF
,<chr>,<chr>,<chr>
1,Pos.R1.C1,Border,Border
2,Pos.R3.C1,Border,Border
3,Pos.R5.C1,Border,Border
4,Pos.R7.C1,Border,Border
5,Pos.R9.C1,Border,Border
6,Pos.R11.C1,Border,Border


In [16]:
MTX_data <- (merge(miniarray_pos_table,MTX_data, by = 'position'))
head(MTX_data)

,position,Miniarray.Gene,Miniarray.ORF,UniqueID,Diploid_sel2_area,Construct_ID,Construct,Tagging,Rep,GeneID,Species,Gene,Species_position,Pos,row,column,auc
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>
1,Pos.R1.C10,Border,Border,pKB11.C.21.Rep2.R1.C10,3224,pKB11.C.21,pKB11,C,2,21,H. sapiens,CSNK1E,H. sapiens(CSNK1E),,1,10,76204.0
2,Pos.R1.C11,Border,Border,pKB11.C.7.Rep1.R1.C11,3013,pKB11.C.7,pKB11,C,1,7,S.mikatae,Smik_6.409,T.blattae(A),A,1,11,79597.5
3,Pos.R1.C11,Border,Border,pKB11.C.22.Rep2.R1.C11,2859,pKB11.C.22,pKB11,C,2,22,H. sapiens,CSNK1A1,H. sapiens(CSNK1A1),,1,11,64398.5
4,Pos.R1.C11,Border,Border,pKB11.C.23.Rep1.R1.C11,2975,pKB11.C.23,pKB11,C,1,23,H. sapiens,CSNK1D,H. sapiens(CSNK1D),,1,11,71993.0
5,Pos.R1.C11,Border,Border,pKB11.C.2.Rep2.R1.C11,3154,pKB11.C.2,pKB11,C,2,2,K.africana,KAFR0K00320,Z.rouxii(Anc),A,1,11,72615.5
6,Pos.R1.C11,Border,Border,pKB11.C.9.Rep1.R1.C11,3001,pKB11.C.9,pKB11,C,1,9,Z.rouxii,ZYRO0G08866g,N.dairenensis(B),Anc,1,11,74252.0


In [17]:
removed_proteins = read.csv("Prot_removed.csv")
head(removed_proteins)
MTX_data2 <-left_join(MTX_data,removed_proteins, by = 'Miniarray.Gene') 

head(MTX_data2)

,Miniarray.Gene,Protein.Removed
,<chr>,<int>
1,YEL063C,1
2,YER026C,1
3,YHR146W,1
4,YKR039W,1
5,YDR508C,1
6,YML123C,1


,position,Miniarray.Gene,Miniarray.ORF,UniqueID,Diploid_sel2_area,Construct_ID,Construct,Tagging,Rep,GeneID,Species,Gene,Species_position,Pos,row,column,auc,Protein.Removed
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<int>
1,Pos.R1.C10,Border,Border,pKB11.C.21.Rep2.R1.C10,3224,pKB11.C.21,pKB11,C,2,21,H. sapiens,CSNK1E,H. sapiens(CSNK1E),,1,10,76204.0,NA
2,Pos.R1.C11,Border,Border,pKB11.C.7.Rep1.R1.C11,3013,pKB11.C.7,pKB11,C,1,7,S.mikatae,Smik_6.409,T.blattae(A),A,1,11,79597.5,NA
3,Pos.R1.C11,Border,Border,pKB11.C.22.Rep2.R1.C11,2859,pKB11.C.22,pKB11,C,2,22,H. sapiens,CSNK1A1,H. sapiens(CSNK1A1),,1,11,64398.5,NA
4,Pos.R1.C11,Border,Border,pKB11.C.23.Rep1.R1.C11,2975,pKB11.C.23,pKB11,C,1,23,H. sapiens,CSNK1D,H. sapiens(CSNK1D),,1,11,71993.0,NA
5,Pos.R1.C11,Border,Border,pKB11.C.2.Rep2.R1.C11,3154,pKB11.C.2,pKB11,C,2,2,K.africana,KAFR0K00320,Z.rouxii(Anc),A,1,11,72615.5,NA
6,Pos.R1.C11,Border,Border,pKB11.C.9.Rep1.R1.C11,3001,pKB11.C.9,pKB11,C,1,9,Z.rouxii,ZYRO0G08866g,N.dairenensis(B),Anc,1,11,74252.0,NA


In [18]:
data =  select(MTX_data2, Construct,Tagging,Rep,GeneID,Species,Gene,Pos,Species_position,Miniarray.Gene,Miniarray.ORF,Protein.Removed,Diploid_sel2_area,auc)

In [19]:
data$Protein.Removed[is.na(data$Protein.Removed)] <- 0

In [20]:
head(data)

,Construct,Tagging,Rep,GeneID,Species,Gene,Pos,Species_position,Miniarray.Gene,Miniarray.ORF,Protein.Removed,Diploid_sel2_area,auc
,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
1,pKB11,C,2,21,H. sapiens,CSNK1E,,H. sapiens(CSNK1E),Border,Border,0,3224,76204.0
2,pKB11,C,1,7,S.mikatae,Smik_6.409,A,T.blattae(A),Border,Border,0,3013,79597.5
3,pKB11,C,2,22,H. sapiens,CSNK1A1,,H. sapiens(CSNK1A1),Border,Border,0,2859,64398.5
4,pKB11,C,1,23,H. sapiens,CSNK1D,,H. sapiens(CSNK1D),Border,Border,0,2975,71993.0
5,pKB11,C,2,2,K.africana,KAFR0K00320,A,Z.rouxii(Anc),Border,Border,0,3154,72615.5
6,pKB11,C,1,9,Z.rouxii,ZYRO0G08866g,Anc,N.dairenensis(B),Border,Border,0,3001,74252.0


In [21]:
write.csv(data,"HRR25_orthologs_PPI_screening_parsed_2022-02-17DEY.csv")